## SCRAP (export en df)

In [70]:
import pandas as pd
import json

# pour exploiter les requêtes
from requests import post

# pour le contrôle des requêtes
from time import sleep, time
from random import randint
from abc import ABCMeta, abstractmethod
from warnings import warn



def convert_arr_2_string(arr, sep):
    """ Convert array to string with separator """
    return sep.join(arr)

def get_term(path):
    """ get term in a path. Otherwise, return 'Non renseigné' """
    if path is not None:
        return path.text.strip()
    return ''

def jprint(obj):
    """ convert array to json """
    # create a formatted string of the Python JSON object
    return json.dumps(obj, sort_keys=True) #, indent=4 

def post_data(root_path, payload, requests, start_time):
    """ post data and get the result  """
    response = post(root_path, json=payload)
    content = response.content
    
    ### pause de 8 à 15s
    sleep(randint(8, 15))
    
    ### afficher les informations sur les requêtes
    requests += 1 # incrémentation du nombre de requête
    elapsed_time = time() - start_time
    
    ### avertir si le code status est différent de 200
    if response.status_code != 200:
        warn('Request: {}; Status code:{}'.format(requests, requests/elapsed_time))
    
    ### stopper quand les requêtes atteignent le quota
    if requests > 200:
        warn('Nombre de requêtes trop important')
        return
    
    try:
        json_data = json.loads(content)
    except:
        json_data = ""
    
    return json_data


class scraping_jobs(metaclass=ABCMeta):
    
    def __init__(self, s_job, type_contract):
        self.s_job = s_job
        self.type_contract = type_contract
    
    @abstractmethod
    def scrap_job(self, dict_jobs, s_job, type_contract):
        pass


class scraping_jobs_apec(scraping_jobs):
    
    #
    def set_code_dpt(self, code_dpt):
        self.code_dpt = code_dpt
    
    #
    def scrap_job(self):
        ### paramètres pris
        # les termes doivent être séparés par ' ' ou '%20'
        param_search_words = self.s_job 
        # le numéro du département
        #param_search_location = self.code_dpt

        
        
        ### pages à parcourir
        pages = [str(i) for i in range(0, 10)]  #### le nombre de pages voulu 
        requests = 0
        start_time = time()
        
        dict_jobs = []
        json_fin ={}
        
        ### parcours des pages
        for page in pages:
            #
            root_path = 'https://www.apec.fr/cms/webservices/rechercheOffre'
            payload = {
                #'lieux': [param_search_location],
                'typeClient': 'CADRE',
                'sorts' : [{
                    'type': 'SCORE',
                    'direction': 'DESCENDING'
                }],
                'pagination': {
                    'range': 20,
                    'startIndex': page
                },
                'activeFiltre': True,
                'pointGeolocDeReference': {
                    'distance': 0
                },
                'motsCles': param_search_words
            }
            
            json_data = post_data(root_path, payload, requests, start_time)
            json_fin.update(json_data)
            

        ### retourne array
        return json_fin
    
s_job = "data"
city = ""
code_dpt = ""
type_contract = ''

arr_jobs = []

## apec
sjapec = scraping_jobs_apec(s_job, type_contract)
sjapec.set_code_dpt(code_dpt)
dict_tmp = sjapec.scrap_job()

if 'offreFilters' in dict_tmp:
    dict_tmp.pop('offreFilters')
    dict_tmp.pop('totalCount')
resultats = dict_tmp.get('resultats', [])

# Create a DataFrame using pandas
df_apec = pd.DataFrame(resultats)


In [71]:
df_apec.shape

(20, 25)

In [64]:
df.head(2)

,id,numeroOffre,intitule,intituleSurbrillance,nomCommercial,lieuTexte,salaireTexte,texteOffre,urlLogo,dateValidation,...,score,offreConfidentielle,secteurActivite,secteurActiviteParent,clientReel,contractDuration,typeContrat,origineCode,indicateurFaibleCandidature,idNomTeletravail
0,172685127,172685127W,Business Data Analyst - Data Wranglers F/H,Business Data Analyst - Data Wranglers F/H,Meteojob,Puteaux - 92,A négocier,Le profil de Data Analyst en Risque de Crédit ...,/media_entreprise/759486/logo_MeteoJob_759486_...,2023-12-22T00:59:04.000+0000,...,41.520866,False,101606,101762,False,0,101888,20382,False,NaN
1,172601750,172601750W,Data Engineer F/H,Data Engineer F/H,CARAZ,Paris 08 - 75,60 - 70 k€ brut annuel,Vous intégrez l'équipe Datahub de l'entreprise...,NaN,2023-12-11T11:46:58.000+0000,...,40.825924,False,101637,101773,True,0,101888,101859,False,20765.0


## Creation des df sous format BDD

In [51]:
# Création du DataFrame df_lieu_apec
df_lieu_apec = pd.DataFrame({
    'id': range(1, len(df_apec) + 1),
    'libelle': df_apec['lieuTexte'],
    'latitude': df_apec['latitude'],
    'longitude': df_apec['longitude'],
    'codePostal': df_apec['lieuTexte'].str[-2:],
    'commune': ''
})
df_lieu_apec.head(1)

,id,libelle,latitude,longitude,codePostal,commune
0,1,Toulouse - 31,43.596013,1.4321019,31,


In [52]:
# Création du DataFrame df_entreprise_apec
df_entreprise_apec = pd.DataFrame({
    'id': range(1, len(df_apec) + 1),
    'nom': df_apec['nomCommercial'],
    'adaptee': ''
})
df_entreprise_apec.head(1)

,id,nom,adaptee
0,1,MP DATA,


In [53]:
# Création du DataFrame df_contrat_apec
df_contrat_apec = pd.DataFrame({
    'id': range(1, len(df_apec) + 1),
    'typeContrat': df_apec['typeContrat'],
    'typeContratLibelle': '',
    'natureContrat': '',
    'dureeTravailLibelle': '',
    'dureeTravailLibelleConverti': '',
    'alternance': ''
})
df_contrat_apec.head(44)

,id,typeContrat,typeContratLibelle,natureContrat,dureeTravailLibelle,dureeTravailLibelleConverti,alternance
0,1,101888,,,,,
1,2,101888,,,,,
2,3,101888,,,,,
3,4,101888,,,,,
4,5,101889,,,,,
5,6,101888,,,,,
6,7,101888,,,,,
7,8,101888,,,,,
8,9,101888,,,,,
9,10,101888,,,,,


In [54]:
df_contrat_apec['typeContrat'].value_counts()

101888    18
101889     2
Name: typeContrat, dtype: int64

In [55]:
# Création du DataFrame df_origine_apec
df_origine_apec = pd.DataFrame({
    'id': range(1, len(df_apec) + 1),
    'origine': 'APEC',
    'adaptee': ''
})
df_origine_apec.head(1)

,id,origine,adaptee
0,1,APEC,


In [56]:
# Création du DataFrame df_qualification_apec
df_qualification_apec = pd.DataFrame({
    'id': range(1, len(df_apec) + 1),
    'qualification': ''
})
df_qualification_apec.head(1)

,id,qualification
0,1,
